In [ ]:
import numpy as np
import pandas as pd
from sklearn import model_selection
import lightgbm as lgb
import os
import shutil
from sklearn.preprocessing import MinMaxScaler
from catboost import CatBoostClassifier

os.system('apt-get install p7zip')
!pip install pyunpack
!pip install patool
from pyunpack.cli import Archive

directory = '/kaggle/working/'
Archive('/kaggle/input/kkbox-music-recommendation-challenge/train.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/test.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/songs.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/members.csv.7z').extractall(directory)
Archive('/kaggle/input/kkbox-music-recommendation-challenge/song_extra_info.csv.7z').extractall(directory)

#print(train['source_screen_name'].value_counts())
#print(train.groupby('msno')['source_type'].nunique())
#import sys
#sys.exit("Error message")
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
songs = pd.read_csv('./songs.csv') #'composer', 'lyricist'
members = pd.read_csv('./members.csv') #
songs_extra = pd.read_csv('./song_extra_info.csv')

print('Data preprocessing...')

dict_count_song_played_train = {k: v for k, v in train['song_id'].value_counts().iteritems()}
dict_count_song_played_test = {k: v for k, v in test['song_id'].value_counts().iteritems()}
def return_number_played(x):
    try:
        return dict_count_song_played_train[x]
    except KeyError:
        try:
            return dict_count_song_played_test[x]
        except KeyError:
            return 0
train['number_of_time_played'] = train['song_id'].apply(lambda x: return_number_played(x))
test['number_of_time_played'] = test['song_id'].apply(lambda x: return_number_played(x))

dict_user_activity = {k:v for k,v in pd.concat([train['msno'] , test['msno']] , axis = 0).value_counts().iteritems()}
def return_user_activity(x):
    try:
        return dict_user_activity[x]
    except KeyError:
        return 0
train['user_activity_msno'] = train['msno'].apply(lambda x: return_user_activity(x))
test['user_activity_msno'] = test['msno'].apply(lambda x: return_user_activity(x))

songs['genre_ids'].fillna('no_genre_id' , inplace= True)

'''
def genre_count(genre):
    if genre == 'no_genre_id':
        return 0
    else :
        return genre.count('|') + 1
songs['genre_ids_count'] = songs['genre_ids'].apply(lambda x: genre_count(x)).astype(np.int8)
'''

#members['registration_year'] = members['registration_init_time'].apply(lambda x: int(str(x)[0:4]))
#members['registration_month'] = members['registration_init_time'].apply(lambda x: int(str(x)[4:6]))
members['registration'] = members['registration_init_time'].apply(lambda x: int(str(x)[0:4]))*12 + members['registration_init_time'].apply(lambda x: int(str(x)[4:6]))
#members['expiration_year'] = members['expiration_date'].apply(lambda x: int(str(x)[0:4]))
#members['expiration_month'] = members['expiration_date'].apply(lambda x: int(str(x)[4:6]))
members['expiration'] = members['expiration_date'].apply(lambda x: int(str(x)[0:4]))*12 + members['expiration_date'].apply(lambda x: int(str(x)[4:6]))
members['membership_month'] = (members['expiration_date'].apply(lambda x: int(str(x)[0:4]))*12 + members['expiration_date'].apply(lambda x: int(str(x)[4:6]))
                               - members['registration_init_time'].apply(lambda x: int(str(x)[0:4]))*12 - members['registration_init_time'].apply(lambda x: int(str(x)[4:6])))

def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan
        
songs_extra['song_year'] = songs_extra['isrc'].apply(isrc_to_year)
songs_extra.drop(['isrc', 'name'], axis = 1, inplace = True)

song_cols = ['song_id', 'genre_ids', 'artist_name', 'song_length', 'language'] #, 'genre_ids_count'
'''
#One-hot encoding
#dummy_features = pd.get_dummies(categorial_data)
#X = pd.concat([numeric_data, dummy_features], axis=1)
songs_genres = np.array(songs['genre_ids'].apply(lambda x: [int(v) for v in str(x).split('|')]))
genres_list = np.unique(songs_genres.ravel())#songs_genres.ravel().unique()
print('Number of genres: ' + str(len(genres_list)))

ohe_genres = np.zeros((len(songs_genres), len(genres_list)))
for s_i, s_genres in enumerate(songs_genres):
    for genre in s_genres:
        g_i = np.where(genres_list == genre) #genres_list.find(genre)
        ohe_genres[s_i, g_i] = 1
        
for g_i, g in enumerate(genres_list):
    songs['genre_' + str(g)] = ohe_genres[:, g_i]
    song_cols.append('genre_' + str(g))
'''
train = train.merge(songs[song_cols], on='song_id', how='left')
test = test.merge(songs[song_cols], on='song_id', how='left')

members_cols = ['msno', 'city', 'bd', 'gender', 'registration', 'expiration', 'membership_month'] #, 'registration_year', 'registration_month', 'expiration_year', 'expiration_month'
train = train.merge(members[members_cols], on='msno', how='left')
test = test.merge(members[members_cols], on='msno', how='left')

train = train.merge(songs_extra, on = 'song_id', how = 'left')
test = test.merge(songs_extra, on = 'song_id', how = 'left')


min_max_scaling = ['number_of_time_played', 'user_activity_msno','membership_month', 'song_length']
for f in min_max_scaling:
    ms = MinMaxScaler()
    train[f] = ms.fit_transform(train[[f]])
    test[f] = ms.transform(test[[f]])

import gc
del members, songs; gc.collect();
 

for col in train.columns:
    if train[col].dtype == object:
        train[col] = train[col].astype('category')
        test[col] = test[col].astype('category')
   
X = train.drop(['target'], axis=1)
y = train['target'].values

X_test = test.drop(['id'], axis=1)
ids = test['id'].values

del train, test; gc.collect();


print('Training LGBM model...')
'''
parameters = {'num_leaves': [200, 250, 300]}
lgb_estimator = lgb.LGBMClassifier(learning_rate = 0.2, max_depth = 10, num_leaves = 250, objective = 'binary', n_estimators=50)
grid = model_selection.GridSearchCV(estimator = lgb_estimator, param_grid = parameters, cv = 2, scoring = 'roc_auc')
grid.fit(X, y, verbose = 5, eval_set = [(X,y)], eval_metric = 'auc')
print(grid.best_params_)
import sys
sys.exit("Error message")
'''

'''
model = lgb.LGBMModel(learning_rate = 0.2, max_depth = 10, num_leaves = 250, objective = 'binary', n_estimators=50)
model.fit(X,y, eval_set = [(X,y)], verbose = 5, eval_metric = 'auc')
importance = pd.DataFrame(index = model.feature_name_ )
importance['0'] = model.feature_importances_ 
print(importance)    
import matplotlib as mpl 
import matplotlib.pyplot as plt
plt.figure(figsize=(21,10), dpi= 100)
plt.barh(importance.index.values, importance['0'])
plt.show()

print('Making predictions and saving them...')
p_test = model.predict(X_test)
subm = pd.DataFrame()
subm['id'] = ids
subm['target'] = p_test
subm.to_csv('submission.csv', index=False)
print('Done!')
'''

'''
def nan_count(data):
    nan = []
    for col in data.columns:
        nan.append([col, data[col].isna().sum()])
    print(pd.DataFrame(nan, columns = ['Column', 'Nan_Count']))
nan_count(test)

train[['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'genre_ids', 'artist_name', 'gender']] = 
train[['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'genre_ids', 'artist_name', 'gender']].fillna('no item')
test[['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'genre_ids', 'artist_name', 'gender']] = 
test[['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'genre_ids', 'artist_name', 'gender']].fillna('no item')

train['language'] = train['language'].fillna(0).apply(lambda x: int(x))
train['city'] = train[ 'city'].fillna(0).apply(lambda x: int(x))
train['song_year'] = train[ 'song_year'].fillna(0).apply(lambda x: int(x))
test['language'] = test['language'].fillna(0).apply(lambda x: int(x))
test['city'] = test['city'].fillna(0).apply(lambda x: int(x))
test['song_year'] = test['song_year'].fillna(0).apply(lambda x: int(x))


print('CatBoost model...')
cb_model = CatBoostClassifier(iterations=100, learning_rate=0.2, depth=10, eval_metric='AUC', random_seed = 42
                              , cat_features = ['msno', 'song_id', 'source_system_tab', 'source_screen_name', 'source_type', 'genre_ids', 'artist_name', 'gender', 'language', 'city'
                                               , 'registration', 'song_year'])
cb_model.fit(X,y, eval_set=(X,y), use_best_model=True, verbose=5)
p_test = cb_model.predict(X_test)
subm = pd.DataFrame()
subm['id'] = ids
subm['target'] = p_test
subm.to_csv('submission.csv', index=False)
print('Done!')
'''